In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from model import model
from DataExtraction import utils
import os
import ujson

In [2]:
# Load, from each model, the results, the model configuration and the evaluation metrics
os.chdir(utils.get_root_dir() + '/model/final_models')
n_models = len([name for name in os.listdir('.') if 'model_' in name])
training_stats, testing_stats, model_results, model_info = {}, {}, {}, {}
for i in range(1, 8):
    print(f'Loading model {i}')
    m = model.load_model(f'model_{i}')
    model_info[m.model_config.model_name] = m.model_config.__dict__
    with open(m.model_config.model_src + '/train/training_stats.json', 'r') as f:
        training_stats[m.model_config.model_name] = ujson.load(f)
    with open(m.model_config.model_src + '/test/losses.json', 'r') as f:
        testing_stats[m.model_config.model_name] = ujson.load(f)
    model_results[m.model_config.model_name] = np.load(m.model_config.model_src + '/test/y_pred.npy'), np.load(m.model_config.model_src + '/test/y_test.npy')
    del m

Loading model 1
Model model_1 created
Loading model 2
Model model_2 created
Loading model 3
Model model_3 created
Loading model 4
Model model_4 created
Loading model 5
Model model_5 created
Loading model 6
Model model_6 created
Loading model 7
Model model_7 created


In [3]:
def get_model_information(model_name, training_stats, testing_stats, model_results, model_info, results_df):
    model_type = model_info['model_type']
    temporal_window = model_info['window_size'] if model_type == 'LSTM' else str(model_info['window_size']) + '/' + str(model_info['subwindow_size'])
    training_time = sum(training_stats['epoch_times'])
    # num_params = sum(model_info['num_parameters'])
    epoch_num = max(training_stats['epoch_num'])
    layers_blocks = model_info['num_layers'] if model_type == 'LSTM' else model_info['xLSTM_config']['num_blocks']
    batch_size = model_info['batch_size']
    dropout = model_info['dropout']
    learning_rate = model_info['lr']
    mse = testing_stats['total']['mse']
    mae = testing_stats['total']['mae']
    rmse = testing_stats['total']['rmse']
    r2 = testing_stats['total']['r2']
    results_df[model_name] = {
        'Type': model_type,
        'Temporal Window': temporal_window,
        'Training Time': training_time,
        # 'Number of Parameters': num_params,
        'Number of Epochs': epoch_num,
        'Layers/Blocks': layers_blocks,
        'Batch Size': batch_size,
        'Dropout': dropout,
        'Learning Rate': learning_rate,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    }

In [4]:
final_df = pd.DataFrame(index=['Type', 'Temporal Window', 'Training Time', 'Number of Epochs', 'Layers/Blocks', 'Batch Size', 'Dropout', 'Learning Rate', 'MSE', 'MAE', 'RMSE', 'R2'])

In [5]:
for model_name in model_info.keys():
    get_model_information(model_name, training_stats[model_name], testing_stats[model_name], model_results[model_name], model_info[model_name], final_df)

In [6]:
final_df

,model_1,model_2,model_3,model_4,model_5,model_6,model_7
Type,LSTM,LSTM,LSTM,LSTM,LSTM,LSTM,LSTM
Temporal Window,None,None,None,None,None,None,None
Training Time,2132.929452,2137.663442,2094.973156,2047.336608,2631.627446,2638.274787,2575.797658
Number of Epochs,200,200,200,200,350,350,350
Layers/Blocks,5,5,5,5,3,3,3
Batch Size,24,24,24,24,64,64,64
Dropout,0.2,0.2,0.2,0.2,0.3,0.3,0.3
Learning Rate,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001
MSE,0.039331,0.042765,0.084506,0.151322,0.019271,0.060586,0.048653
MAE,0.143567,0.161574,0.237904,0.317241,0.099551,0.19361,0.178644
